In [1]:
import torch 
from torch.autograd import Variable
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from ast import literal_eval
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
torch.manual_seed(43)

In [2]:
# training_set = pd.read_csv('../CSV_files/BALL/dataBall_SIFT-sensors.csv')
training_set = pd.read_csv('../CSV_files/KITTI/SIFT/kitti_SIFTDepthless-sensorsALL.csv')
dataSet_len = len(training_set)
print(f"number of frames: {dataSet_len}")
featuresAreCoordinates = False
training_set.describe()

number of frames: 1242


,x_displacement,y_displacement
count,1242.000000,1242.000000
mean,0.512738,-0.020356
std,0.747939,0.643801
min,-4.269464,-1.247837
25%,0.255773,-0.246604
50%,0.555293,-0.002406
75%,1.070713,0.125856
max,2.550284,7.879498


In [3]:
distance_sensorData = [literal_eval(i) for i in training_set['sensors_e']]
x_sensorData = [literal_eval(i) for i in training_set['sensors_x']]
y_sensorData = [literal_eval(i) for i in training_set['sensors_y']]
if featuresAreCoordinates:
    coordinates = [literal_eval(i) for i in training_set['coordinates_src']]
    coordinatesDST = [literal_eval(i) for i in training_set['coordinates_dst']]
    for i in range(len(coordinates)):
        coordinates[i].extend(coordinatesDST[i])

In [4]:
numberOfSensorsList = [len(i) for i in distance_sensorData]
meanOfSensors = sum(numberOfSensorsList)//len(numberOfSensorsList)
maxNumberSensors = max(numberOfSensorsList)
minNumberSensors = min(numberOfSensorsList)

print(f"max amount of sensors: {maxNumberSensors}")
print(f"min amount of sensors: {minNumberSensors}")
print(f"mean amount of sensors: {meanOfSensors}")

max amount of sensors: 130
min amount of sensors: 130
mean amount of sensors: 130


In [5]:
def standarizeSensors(list_sensors, threshold):
    while len(list_sensors) != threshold:
        if len(list_sensors) < threshold:
            list_sensors.append(0.0)
        elif len(list_sensors) > threshold:
            list_sensors = list_sensors[:threshold]
    return list_sensors

In [6]:
# trim to normalized number of sensors
distance_sensorData = [standarizeSensors(i, meanOfSensors) for i in distance_sensorData]
x_sensorData = [standarizeSensors(i, meanOfSensors) for i in x_sensorData]
y_sensorData = [standarizeSensors(i, meanOfSensors) for i in y_sensorData]
if featuresAreCoordinates:
    coordinates = [standardizeSensors(i, meanOfSensors*4) for i in coordinates]

In [7]:
# normalize feature information
scFeatures = MinMaxScaler()
distance_sensorData = scFeatures.fit_transform(distance_sensorData)
x_sensorData = scFeatures.fit_transform(x_sensorData)
y_sensorData = scFeatures.fit_transform(y_sensorData)

In [8]:
coordinates_depth = 4
displacement_depth = 3

if featuresAreCoordinates:
    feature_data = np.array(coordinates)
    feature_data = feature_data.reshape(feature_data.shape[0], -1)
    print(feature_data.shape)
else: 
    feature_data = []
    for i in range(dataSet_len):
        feature_data.append([x_sensorData[i], y_sensorData[i], distance_sensorData[i]])
    feature_data = np.array(feature_data)
    print(feature_data.shape)
    feature_data = feature_data.reshape(feature_data.shape[0], -1)
    print(feature_data.shape)

(1242, 3, 130)
(1242, 390)


In [9]:
# iloc[row_selection, column_selection]
# can be used to separate between feature data and targets
# feature data: data we watn to learn from
# targets: desired outcome
sc = MinMaxScaler()
df_target = pd.DataFrame({'x_displacement':target_dataX,'y_displacement':target_dataY})
target_data = sc.fit_transform(df_target)

plt.plot( target_data, label = 'target volume')
plt.legend()
plt.show()

## Linear regression model

In [10]:
target_data_reshaped = np.array(target_data)[:, np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size = 0.3, shuffle= True)

dataX = torch.Tensor(feature_data)
dataY = torch.Tensor(target_data)

trainX = torch.Tensor(X_train)
trainY = torch.Tensor(y_train)

testX = torch.Tensor(X_test)
testY = torch.Tensor(y_test)

In [12]:
class LinearRegressionModel(torch.nn.Module): 
    def __init__(self, input_dim, output_dim): 
        super(LinearRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(input_dim, output_dim) 
    def forward(self, x): 
        y_pred = self.linear(x) 
        return y_pred 

In [13]:
learning_rate = 0.01
epochs = 100
output_size = 2
if featuresAreCoordinates:
    input_size = meanOfSensors*coordinates_depth
else:
    input_size = meanOfSensors*displacement_depth

model = LinearRegressionModel(input_size, output_size)

In [14]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 

## Train model

In [15]:
model.train()
for epoch in range(epochs): 
    # Forward pass: Compute predicted y by passing  
    # x to the model 
    output = model(trainX) 
    # Zero gradients, perform a backward pass,  
    optimizer.zero_grad()  
    # Compute and print loss 
    loss = criterion(output, trainY) 
  
    # update the weights. 
    loss.backward() 
    optimizer.step() 
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
    if epoch == epochs-1:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Epoch: 0, loss: 0.36752
Epoch: 99, loss: 0.00775


In [16]:
model.eval() # sets the model to evaluation mode
test_predict = model(testX)
real_loss = criterion(test_predict, testY)
print(f"Test loss: {real_loss.item()}")

train_predict = model(dataX)
data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()


# create empty table with 12 fields
trainPredict_dataset_like = np.zeros(shape=(len(target_data), 2) )
# put the predicted values in the right field
trainPredict_dataset_like[:, 0] = data_predict[:,0]
# inverse transform and then select the right field
data_predict = sc.inverse_transform(trainPredict_dataset_like)[:,0]
trainPredict_dataset_like[:, 0] = dataY_plot[:,0]
dataY_plot = sc.inverse_transform(trainPredict_dataset_like)[:,0]

plt.plot(dataY_plot, label = "real volume")
plt.plot(data_predict, label = "predicted volume")
plt.suptitle('Volume Prediction')
plt.legend()
plt.show()


In [17]:
#Saving a checkpoint
# torch.save(model.state_dict(), 'MODELS/BALL/ball-LinearRegression-SIFTDisplacement.pth')